In [6]:

import ipympl

from lsst.pipe.tasks.sourceClassification import SourceClassificationTask
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import glob
import sncosmo
import pickle
from astropy.table import Table
import re
import astropy.coordinates as coord
import astropy.units as u

import numpy as np
import glob

import Utils.DiaSourceTools as DSTools
import Utils.SourcePlotting as SP
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import clear_output

In [7]:
season_catalogs = []
visit_refs = []
in_directory = "season_catalogs/6sigma/"
c_files = glob.glob(in_directory+"*.fits")
for c_f in c_files:
    catalog = afwTable.SourceCatalog.readFits(c_f)
    visit =  c_f.split("/")[-1].split("-")[0]
    fil  = c_f.split("/")[-1].split("-")[-1].split(".")[0]
    if fil == "r":
        season_catalogs.append(catalog)
        visit_refs.append(visit)
        
        

In [8]:
def multi_match_catalogs(catalog_list, data_refs):
    multi_matches = None

    for catalog, data_ref in zip(catalog_list, data_refs):

            if multi_matches is None and len(catalog)>0:
                multi_matches = afwTable.MultiMatch(catalog[0].schema, {'visit':int}, radius=afwGeom.Angle(1./3600., afwGeom.degrees))
            if multi_matches is not None:
                multi_matches.add(catalog, {'visit':int(data_ref)})

    results = multi_matches.finish(removeAmbiguous=False)  
    return results



In [9]:
results = multi_match_catalogs(season_catalogs, visit_refs)
print len(results)

188202


In [10]:
light_curves=DSTools.get_light_curves_from_multimatch_results(results)
print len(light_curves)

57598


In [11]:
t_light_curves = DSTools.threshold_light_curves(light_curves, 10)
print len(t_light_curves)

2877


In [18]:
reload(DSTools)

<module 'Utils.DiaSourceTools' from 'Utils/DiaSourceTools.py'>

In [19]:
lcs1 = []
for t_light_curve in t_light_curves:
    lcs1.append(DSTools.build_lightcurve_tuple(t_light_curve, "base_CircularApertureFlux_3_0_flux"))

In [63]:
lcs2 = []
for t_light_curve in t_light_curves:
    lcs2.append(DSTools.build_lightcurve(t_light_curve, "base_CircularApertureFlux_4_5_flux"))

In [64]:
lcs3 = []
for t_light_curve in t_light_curves:
    lcs3.append(DSTools.build_lightcurve(t_light_curve, "ip_diffim_PsfDipoleFlux_pos_flux"))

In [10]:


classification_dict = {0:"Dipole type I", 1: "Dipole type II", 3:"Fringe", 4: "Artifact", 5:"Negative", 6:"Positive", 7:"Point Positive"}
SP.plot_object_distribution(lcs, classification_dict)

In [11]:
import re
import astropy.coordinates as coord
import astropy.units as u

filter='r'

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table9.dat.txt','r')
data_elems = f.read()
elems = re.findall('^(06D3.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table10.dat.txt','r')
data = f.read()

snls_array = []
for sn in elems:
    c = coord.SkyCoord(sn[-1], unit=(u.hourangle, u.deg))
    
    m = re.findall('^'+str(sn[0])+'\\|('+filter+')\\|(.*?)\\|(.*?)\\|(.*?)$', data, re.MULTILINE)

    snls_lc = DSTools.build_light_curve_from_snls_file(m, c)
    plt.errorbar(snls_lc['mjd'], snls_lc['flux'], yerr=snls_lc['flux_error'], fmt='.', color='blue')
    snls_array.append(snls_lc)
    plt.show()

In [12]:
matches = DSTools.match_control_group(lcs,snls_array)
float(len(matches))/float(len(snls_array))

0.11764705882352941

In [13]:
import lsst.daf.persistence as dafPersist

DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output") 


In [26]:
import lsst.daf.persistence as dafPersist

DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output") 

In [24]:
def get_stamps_in_lc(lc):
    ccds= range(0,36)
    stamps = []
    for point in lc:

        for ccd in ccds: 
            if butler.datasetExists("deepDiff_differenceExp",dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd}):
                diffExp = butler.get("deepDiff_differenceExp", dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd})
                box = diffExp.getBBox()
                wcs = diffExp.getWcs()
                coord = afwGeom.Point2I(wcs.skyToPixel(point["ra"], point["dec"]))
                if box.contains(coord):
                    break


        #calexp = butler.get("calexp", dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd})
        calexp = butler.get("calexp", dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd})



        Center = afwGeom.Point2I(wcs.skyToPixel(point["ra"], point["dec"]))
        Begin = afwGeom.Point2I(Center.getX() - 10, Center.getY() - 10)
        End = afwGeom.Point2I(Center.getX() + 10, Center.getY() + 10)

        newBox = afwGeom.Box2I(Begin,End) 
        try:
            cal_stamp = calexp.Factory(calexp, newBox, True)
            stamp = diffExp.Factory(diffExp,newBox, True)
            stamps.append((stamp, cal_stamp))
        except Exception, e:
            print e
            stamps.append((None, None))
            pass
        
    return stamps
    

In [22]:
lc = lcs1[45]
print len(lc)

48


In [27]:

stamps = get_stamps_in_lc(lc)

In [28]:
plt.close("all")
fig = plt.figure(figsize=(10,5))
fig2 = plt.figure(figsize=(10,10))
plt.figure(fig.number)
#plt.ylim(ymin=-75000, ymax=100000)
tolerance = 5 # points
plt.errorbar(lc["mjd"],lc["flux"], yerr=lc["flux_error"],fmt='.', color='blue', picker=tolerance)

fig.canvas.mpl_connect('pick_event', lambda event: on_pick(event, stamps , fig2.number))
plt.show()

In [20]:
def on_pick(event, stamps, fig_number):
    
    clear_output()
    
    plt.figure(fig_number)
    plt.clf()
    artist = event.artist
    
    xmouse, ymouse = event.mouseevent.xdata, event.mouseevent.ydata
    x, y = artist.get_xdata(), artist.get_ydata()
    ind = event.ind

    stamp, c_stamp = stamps[ind[0]]


    plt.subplot(2,2,1)
    img_arr, mask_arr, var_arr = stamp.getMaskedImage().getArrays()
    w,h = stamp.getWidth(), stamp.getHeight()  
    plt.imshow(img_arr, origin='lower', vmin=img_arr.min(), vmax=img_arr.max(), cmap='gray', extent=(0,w-1, 0, h-1), interpolation='none')
    plt.subplot(2,2,3)
    plt.hist(img_arr.flat, 100)
    
    plt.subplot(2,2,2)
    img_arr, mask_arr, var_arr = c_stamp.getMaskedImage().getArrays()
    w,h = c_stamp.getWidth(), c_stamp.getHeight()
    plt.imshow(img_arr, origin='lower', vmin=img_arr.min(), vmax=img_arr.max(), cmap='gray', extent=(0,w-1, 0, h-1), interpolation='none')
    plt.subplot(2,2,4) 
    plt.hist(img_arr.flat, 100)
        
    plt.show()
    

In [109]:
stamp, c_stamp = stamps[0]
img_arr, mask_arr, var_arr = stamp.getMaskedImage().getArrays()

In [107]:
plt.hist(img_arr.flat, 100)
plt.show()

In [111]:
img_arr, mask_arr, var_arr = c_stamp.getMaskedImage().getArrays()
plt.hist(img_arr.flat, 100)
plt.show()